In [4]:
import numpy as np
import yfinance as yf
import pandas as pd
import talib as ta

In [5]:
sp500_df = pd.read_csv('./data/Stocks in the SP 500 Index.csv')
sp500_df.head(3)

,Symbol,Description,Category2,Category3,GICS Sector,Market cap,Dividend yield,Price to TTM earnings,Price to TTM sales,Price to book value,Action
0,AAPL,Apple Inc,Common stocks,Large cap,Information Technology,"$2,815,919,544,550",0.51%,27.41,7.44,38.31,Analyze
1,MSFT,Microsoft Corp,Common stocks,Large cap,Information Technology,"$2,245,311,495,186",0.79%,30.62,12.14,13.62,Analyze
2,GOOG,Alphabet Inc Class C,Common stocks,Large cap,Communication Services,"$1,856,551,479,248",0.00%,10.99,7.21,3.32,Analyze


In [6]:
def add_ratio1 (df):
    df['Ratio1']=((df['Close']+df['Open'])/2-df['Low'])/(df['High']-df['Low'])

def add_ratio2 (df):
    df['Ratio2']=(df['Close']-df['Open'])/(df['High']-df['Low'])

def add_avg (df):
    df['avg']=(df['Open']+df['Close']+df['High']+df['Low'])/4

def add_trend_tmr_up (df):
    df['avg_tmr']=np.concatenate((df['avg'].to_numpy()[1:],[None]))
    df['trend_tmr_up'] = np.where(df['avg_tmr']-df['avg']>0, True, False)
    #######################
    df.dropna(inplace=True)
    #######################
    del df['avg_tmr']

In [11]:
def sp_500_stocks (timeframe):
    df_all=pd.DataFrame()
    #for i in sp500_df.Symbol:
    for i in sp500_df.Symbol[0:1]:
        stk = yf.Ticker(i)
        df = stk.history(period=timeframe)
        add_ratio1(df)
        add_ratio2(df)
        add_avg (df)
        add_trend_tmr_up (df) #dropna happens here
        df['Name']=i
        df_all=pd.concat([df_all, df])
    return df_all

In [12]:
x = sp_500_stocks('1y')
print(x.shape)
x.head(3)

(252, 12)


,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ratio1,Ratio2,avg,trend_tmr_up,Name
Date,,,,,,,,,,,,
2021-02-24,124.201809,124.818141,121.507822,124.609383,111039900,0.0,0,0.875376,0.123122,123.784289,False,AAPL
2021-02-25,123.943339,125.712821,119.827801,120.275139,148199500,0.0,0,0.387669,-0.623311,122.439775,False,AAPL
2021-02-26,121.865692,124.112342,120.483906,120.543556,164560400,0.0,0,0.198631,-0.364382,121.751374,True,AAPL


In [ ]:
x.to_csv('./data/sp500_2y.csv')